In [1]:
# Libaries
import re
import pymongo

In [2]:
# Database connection
""" in case you want to create your on database, just change this 3 variables according to your database. 
Database and collections will be created automatically."""
username = "globaladmin"
password = "UEKaeSc8Q3LK5naw"
db_url = "cluster1.etdy2wm.mongodb.net"

# database string
CNX_STR = f"mongodb+srv://{username}:{password}@{db_url}/?connectTimeoutMS=50000"
client = pymongo.MongoClient(CNX_STR)

# heck if the database exists
if "manage2sail" in client.list_database_names():
    print("Database exists!")
    db = client.manage2sail
else:
    print("Database does not exist!")

Database exists!


In [3]:
# clean results for results with fleets/ groups
def clean_results(results_by_class):
    for class_name, class_data in results_by_class.items():
        if isinstance(class_data, str) and class_data == "no results":
            # Skip cleaning if the class_data is "no results"
            continue
        if isinstance(class_data, dict) and 'results' in class_data and isinstance(class_data['results'], list):
            for entry in class_data['results']:
                if isinstance(entry, dict):
                    # Identify and process race results
                    keys_to_remove = []
                    new_entries = {}
                    for key, value in entry.items():
                        if isinstance(value, str) and '\n' in value:
                            race_results = value.split('\n')
                            cleaned_race_results = []
                            buffer = ""
                            for race_result in race_results:
                                race_result = race_result.strip()
                                if race_result.isdigit() or '.' in race_result:
                                    if buffer:
                                        cleaned_race_results.append(buffer.strip())
                                    buffer = race_result
                                else:
                                    buffer += f" {race_result}"
                            if buffer:
                                cleaned_race_results.append(buffer.strip())

                            for i, race_result in enumerate(cleaned_race_results):
                                new_entries[f'R{i + 1}'] = race_result
                            keys_to_remove.append(key)

                    # Remove original concatenated race result entries
                    for key in keys_to_remove:
                        del entry[key]

                    # Add the new individual race results
                    entry.update(new_entries)
                else:
                    print(f"Unexpected entry type: {type(entry)}, entry: {entry}")
        else:
            print(f"Unexpected class_data structure for class {class_name}: {class_data}")
    return results_by_class

In [4]:
def trim_whitespace(document):
    def trim_value(value):
        if isinstance(value, str):
            return value.strip()
        elif isinstance(value, dict):
            return {k: trim_value(v) for k, v in value.items()}
        elif isinstance(value, list):
            return [trim_value(item) for item in value]
        else:
            return value

    return {k: trim_value(v) for k, v in document.items()}

In [5]:
def replace_line_breaks_and_spaces(results_by_class):
    for class_name, class_data in results_by_class.items():
        if isinstance(class_data, str) and class_data == "no results":
            # Skip cleaning if the class_data is "no results"
            continue
        if isinstance(class_data, dict) and 'results' in class_data and isinstance(class_data['results'], list):
            for entry in class_data['results']:
                if isinstance(entry, dict):
                    for key, value in entry.items():
                        if isinstance(value, str):
                            # Replace line breaks with spaces
                            value = value.replace('\n', ' ')
                            # Replace multiple spaces with a single space
                            value = re.sub(r'\s+', ' ', value)
                            entry[key] = value
    return results_by_class

In [6]:
def update_documents():
    collection = db.events
    documents = list(collection.find())

    for document in documents:
        if 'resultsByClass' in document:
            print(f"Processing document {document['_id']}")
            original_document = document.copy()
            document = trim_whitespace(document)
            document['resultsByClass'] = replace_line_breaks_and_spaces(document['resultsByClass'])
            cleaned_results = clean_results(document['resultsByClass'])
            document['resultsByClass'] = cleaned_results
            
            if original_document != document:
                collection.update_one({'_id': document['_id']}, {'$set': document})
                print(f"Document {document['_id']} updated successfully.")

In [7]:
if __name__ == "__main__":
    update_documents()

Processing document 6679a1780fd5641fb351fc31
Document 6679a1780fd5641fb351fc31 updated successfully.
Processing document 6679a1780fd5641fb351fc32
Processing document 6679a1780fd5641fb351fc33
Processing document 6679a1780fd5641fb351fc34
Processing document 6679a1780fd5641fb351fc36
Processing document 6679a1780fd5641fb351fc38
Processing document 6679a1780fd5641fb351fc39
Processing document 6679a1780fd5641fb351fc41
Processing document 6679a1780fd5641fb351fc44
Processing document 6679a1780fd5641fb351fc47
Processing document 6679a1780fd5641fb351fc4b
Document 6679a1780fd5641fb351fc4b updated successfully.
Processing document 6679a1780fd5641fb351fc4d
Processing document 6679a1780fd5641fb351fc53
Document 6679a1780fd5641fb351fc53 updated successfully.
Processing document 6679a1780fd5641fb351fc58
Processing document 6679a1780fd5641fb351fc5a
Document 6679a1780fd5641fb351fc5a updated successfully.
Processing document 6679a1780fd5641fb351fc5e
Document 6679a1780fd5641fb351fc5e updated successfully.


In [7]:
# Swiss Sailing League goes Punat (Croatia) - Regatta
